In [1]:
! [ -e /content ] && pip install -Uqq fastbook kaggle waterfallcharts treeinterpreter dtreeviz

In [2]:
from fastbook import *
from pandas.api.types import is_string_dtype, is_numeric_dtype, is_categorical_dtype
from fastai.tabular.all import *
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score, make_scorer
from sklearn.model_selection import cross_val_score
from IPython.display import Image, display_svg, SVG
import xgboost as xgb
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, accuracy_score
import numpy as np

max_d = 5
num_est = 80
validation_lookback = 14
train_start_days_past = 94
should_drop_prior_gst = False
lr = 0.1

In [3]:
path = Path.cwd()
Path.BASE_PATH = path
dsets_path = path.parent.parent/'fastai/datasets'
df = pd.read_csv(dsets_path/'osrs_trades.csv', low_memory=False)
dep_var = 'gst'
make_date(df, 'timestamp')
df = add_datepart(df, 'timestamp')
df = df.drop('id', axis=1)
if should_drop_prior_gst:
    df = df.drop('prior_gst', axis=1)
    
procs = [Categorify, FillMissing]
maxDay = df['timestampDayofyear'].max()

minCond = (df.timestampDayofyear>(maxDay - train_start_days_past)) # day 117 is when I fixed date recording on completed_trades
maxCond = (df.timestampDayofyear<=(maxDay - validation_lookback))
train_idx = np.where((maxCond) & (minCond))[0]
valid_idx = np.where(~maxCond)[0]

splits = (list(train_idx),list(valid_idx))
cont,cat = cont_cat_split(df, 1, dep_var=dep_var)
to = TabularPandas(df, procs, cat, cont, y_names=dep_var, splits=splits)
save_pickle(path/'preprocessed_train_valid.pkl', to)

/home/tristan/Documents/dev/ML_exercises/random_forest/default/lib/python3.12/site-packages/fastai/tabular/core.py:23: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df[date_field] = pd.to_datetime(df[date_field], infer_datetime_format=True)
/home/tristan/Documents/dev/ML_exercises/random_forest/default/lib/python3.12/site-packages/fastai/tabular/core.py:312: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] 

In [4]:
xs,y = to.train.xs,to.train.y
valid_xs,valid_y = to.valid.xs,to.valid.y

In [5]:
xg_reg = xgb.XGBRegressor(objective='reg:squarederror', colsample_bytree=0.3, learning_rate=lr,
                          max_depth=max_d, alpha=10, n_estimators=num_est)
xg_reg.fit(xs, y)

y_pred = xg_reg.predict(valid_xs)
mse = mean_squared_error(valid_y, y_pred)

def accuracy(pred, y): return round(abs(y - pred).mean(), 2)
def m_accuracy(preds, y): return accuracy(preds, y)

accuracy = m_accuracy(y_pred, valid_y)
rmse = np.sqrt(mse)

rmse, accuracy

(89.577156, 44.35)